In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd


# Load Data

In [3]:
df = pd.read_csv("../../data/AmesHousing.csv")


In [4]:
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()


In [5]:
k = 10000

In [6]:
raw_y = df["SalePrice"].copy()
X = df.drop("SalePrice", axis=1).copy()

In [7]:
X = X.head(k)
raw_y = raw_y.loc[X.index].copy()

In [8]:
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

In [9]:
y = (raw_y >= raw_y.quantile(0.5)).astype(int).copy()

In [10]:
X[cat_cols] = X[cat_cols].astype("category")

# PRISM

In [11]:
from prism import *

In [12]:
%autoreload 2

In [13]:
%%time
sorter = CategoricalSorter(scoring_function=woe_scoring_func)
encoder = SortedOrdinalEncoder(sorter)
bin_calculator = NBinsCalculator(strategy=FreedmanDiaconisStrategy(), max_bins=20)
discretizer = PandasQCutDiscretizer(bin_calculator=bin_calculator)
evaluator = BinaryRuleEvaluator(score_strategies=[WoEScore(), RecallScore(), PrecisionScore()])
rule_filter = BinaryRuleFilter(min_recall=0.1)
prism = PrismRules(encoder, discretizer, evaluator)
rules = prism.find_rules(X, y, rule_filter, max_depth=3)

Starting to find rules...
Path: (`Neighborhood` > 23.0) with score: 4.532199021266578 0.19313593539703902 0.9896551724137931
Path: (`Neighborhood` > 23.0) and (`Lot Area` > 4800.0) with score: 28.613951121782094 0.18506056527590847 1.0
Path: (`Neighborhood` <= 23.0) and (`Overall Qual` > 7.0) with score: 3.8753199279059487 0.16689098250336473 0.9802371541501976
Path: (`Neighborhood` <= 23.0) and (`Overall Qual` > 7.0) and (`Exterior 2nd` > 4.0) with score: 5.382975146030107 0.15074024226110364 0.9955555555555555
Path: (`Neighborhood` <= 23.0) and (`Overall Qual` > 7.0) and (`Exterior 2nd` > 4.0) and (`Mo Sold` > 2.0) with score: 28.265038183178852 0.13055181695827725 1.0
Path: (`Neighborhood` <= 23.0) and (`Overall Qual` <= 7.0) and (`Neighborhood` > 19.0) with score: 2.8709175071761557 0.14670255720053835 0.9478260869565217
Path: (`Neighborhood` <= 23.0) and (`Overall Qual` <= 7.0) and (`Neighborhood` > 19.0) and (`Open Porch SF` > 16.0) with score: 3.9603131407393413 0.10901749663526

In [ ]:
rules

In [ ]:
#remove redudant rules(`Overall Qual` <= 7.0) and (`Overall Qual` <= 6.0)
#convert (`Overall Qual` <= 7.0) and (`Overall Qual` > 6.0) to (`Overall Qual` == 7.0)
#convert a < 6 and a >= 3 to 3 <= a < 6

In [ ]:
#display chains - start with root
# on the next one - the root should be the main rule with stats

In [ ]:
# add stats to the rules
# add nice prints to rules